In [2]:
import numpy as np

<h2 align="center">BackProp and Optimizers</h2>
<img src="img/bp.png" width="600">

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [114]:
class Module(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True
    """
    Basically, you can think of a module as of a something (black box) 
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`: 
        
        output = module.forward(input)
    
    The module should be able to perform a backward pass: to differentiate the `forward` function. 
    More, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule. 
    
        gradInput = module.backward(input, gradOutput)
    """
    
    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(input)

    def backward(self, input, gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.
        
        This includes 
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput
    

    def updateOutput(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.
        
        Make sure to both store the data in `output` field and return it. 
        """
        
        # The easiest case:
            
        # self.output = input 
        # return self.output
        
        pass

    def updateGradInput(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own input. 
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.
        
        The shape of `gradInput` is always the same as the shape of `input`.
        
        Make sure to both store the gradients in `gradInput` field and return it.
        """
        
        # The easiest case:
        
        # self.gradInput = gradOutput 
        # return self.gradInput
        
        pass   
    
    def accGradParameters(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass
    
    def zeroGradParameters(self): 
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass
        
    def getParameters(self):
        """
        Returns a list with its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
        
    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
    
    def training(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True
    
    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False
    
    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Module"

# Sequential container

**Define** a forward and backward pass procedures.

In [115]:
class Sequential(Module):
    """
         This class implements a container, which processes `input` data sequentially. 
         
         `input` is processed by each module (layer) in self.modules consecutively.
         The reslr ulting array is called `output`. 
    """
    
    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []
   
    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)

    def updateOutput(self, input):
        """
        Basic workflow of FORWARD PASS:
        
            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})   
            
        Just write a little loop.
        """

        # Your code goes here. ################################################
        
        y_i = input
        for i in np.arange(len(self.modules)):
            y_i = self.modules[i].forward(y_i)
#             print('forward: y_i ', y_i)
        
        self.output = y_i
        return self.output

    def backward(self, input, gradOutput):
        """
        Workflow of BACKWARD PASS:
            
            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)   
            gradInput = module[0].backward(input, g_1)   
                
        !!!
                
        To each module you need to provide the input, module saw while forward pass, 
         it is used while computing gradients. 
        Make sure that the input for `i-th` layer the output of `module[i]` 
         (just the same input as in forward pass) 
         and NOT `input` to this Sequential module. 
        
        !!!
        
        """
        
        # Your code goes here. ################################################
        g_i = gradOutput
        n = len(self.modules)
        for i in np.arange(n - 1, 0, -1):
            g_i = self.modules[i].backward(self.modules[i-1].output, g_i)
#             print('backward: g_i =', g_i)
        
        self.gradInput = self.modules[0].backward(input, g_i)      
        return self.gradInput

    def zeroGradParameters(self): 
        for module in self.modules:
            module.zeroGradParameters()
    
    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]
    
    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]
    
    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string
    
    def __getitem__(self, x):
        return self.modules.__getitem__(x)

# Layers

- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

Про logloss [тут](https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b)

In [104]:
class Linear(Module):
    """
    A module which applies a linear transformation 
    A common name is fully-connected layer.
    
    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()
       
        # This is a nice initialization
        # Also, you can make b within W, it is easier.
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size=(n_in, n_out))
        self.b = np.random.uniform(-stdv, stdv, size = n_out)
        
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        
        # [bs x n_out] = [bs x n_in] * [n_in x n_out]
        self.output = input @ self.W + self.b
#         print('#FUCKRKN', self.b.shape,'inp=' ,input.shape, 'w=', self.W)
        return self.output 
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
               
        """updateGradInput пробрасывает градиент назад
        приходит gradOutput = dL/dy. тут вычисляем dL/dx и 
        возвращаем его в качестве gradInput (х - вход слоя, у - выход)
        
        dL/dx = dL/dy * dy/dx
        dy/dx = W
        """
        
        # [bs x n_in] = [bs x n_out] *  [n_out x n_in]
        self.gradInput = gradOutput @ self.W.T
        return self.gradInput
    
    def accGradParameters(self, input, gradOutput):
        # Your code goes here. ################################################
        
        """
        запоминаем dL/dW и dL/db
        dL/dW = dy/dW * dL/dy = x * dL/dy
        dL/db = dy/db * dL/dy = [1] * dL/dy
        """ 
        # [n_in x n_out] = [n_in x bs] * [bs x n_out]
        self.gradW = input.T @ gradOutput
        self.gradb = gradOutput.sum(axis=0)
    
    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)
        
    def getParameters(self):
        return [self.W, self.b]
    
    def getGradParameters(self):
        return [self.gradW, self.gradb]
    
    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

This one is probably the hardest but as others only takes 5 lines of code in total. 
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/

    Поле для кспериментов для `SoftMax`

In [3]:
batch_size = 2
n = 3
v = np.array([[1, 2, 3], [2, 3, 4]])
# print('v = ', v)

f1 = np.repeat(np.eye(n).reshape((1, n, n)), batch_size, axis=0) # генерируем bs единичных матриц
f2 = np.repeat(v, n, axis=0).reshape((batch_size, n, n)) # генерируем bs матриц, каждая из которых-
f3 = np.repeat(v.ravel().reshape((-1, 1)), n, axis=1).reshape((batch_size, n, n))


f4 = (f2 * (f1 - f3)) # считаем все bs матриц (умножение element wise)
# print('f4=', f4)
f5 = v

# print('f1=',f1)
# print('f2=',f2)
# print('f3=', f3)
# print('f4=', f4)
# print('f5=', f5)

f6 = (f4 @ f5.reshape((batch_size, n, 1))).reshape((batch_size, n))
# print('f6=', f6)

In [88]:
class SoftMax(Module):
    def __init__(self):
         super(SoftMax, self).__init__()
    
    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        
        # Your code goes here. ################################################
        total = np.exp(self.output).sum(axis=1)
        
        self.output = np.exp(self.output) / np.repeat(total.reshape((1, -1)), input.shape[1], axis=0).T
        
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        """
        dL/dx = dy/dx * dL/dy
        
        dy/dx = см ссылку
        
        """
        
        S = self.output
        batch_size, n = S.shape
        
        f1 = np.repeat(np.eye(n).reshape((1, n, n)), batch_size, axis=0)
        f2 = np.repeat(S, n, axis=0).reshape((batch_size, n, n))
        f3 = np.repeat(S.reshape((-1, 1)), n, axis=1).reshape((batch_size, n, n))
        f4 = (f2 * (f1 - f3))
        f5 = gradOutput
        f6 = (f4 @ f5.reshape((batch_size, n, 1))).reshape((batch_size, n))
        self.gradInput = f6
        return self.gradInput
    
    def __repr__(self):
        return "SoftMax"

Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. 

This is a very cool regularizer. In fact, when you see your net is overfitting try to add more dropout.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch). When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [2]:
from scipy.stats import bernoulli

In [95]:
bernoulli.rvs(0.5, size=100).reshape((10, 10))

array([[0, 1, 0, 0, 1, 1, 1, 1, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 1, 0, 1, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 1, 1, 0, 1],
       [0, 0, 1, 0, 0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 0, 1, 1, 1, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 0, 1, 1, 0, 1, 1],
       [1, 0, 0, 1, 1, 1, 0, 0, 1, 0]])

In [96]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        
        self.p = p
        self.mask = None
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        
        self.output = input
        if self.training:
            batch_size, n = input.shape
            self.mask = bernoulli.rvs(self.p, size=batch_size * n).reshape((batch_size, n))
            self.output = self.output * self.mask
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        
        self.gradInput = self.mask * gradOutput
        
        return self.gradInput
        
    def __repr__(self):
        return "Dropout"

# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**): 

In [97]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()
    
    def updateOutput(self, input):
        self.output = np.maximum(input, 0)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput
    
    def __repr__(self):
        return "ReLU"

Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Experiment with slope. 

In [98]:
class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        super(LeakyReLU, self).__init__()
            
        self.slope = slope
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.maximum(input, self.slope * input)
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = np.multiply(gradOutput, (input > 0) + self.slope * (input <= 0))
        return self.gradInput
    
    def __repr__(self):
        return "LeakyReLU"

# Criterions

Criterions are used to score the models answers. 

In [99]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None
        
    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function 
            associated to the criterion and return the result.
            
            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result. 

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(input, target)
    
    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput   

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.

In [101]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()
        
    def updateOutput(self, input, target):
        self.output = np.sum(np.power(input - target, 2)) / input.shape[0]
        return self.output 
 
    def updateGradInput(self, input, target):
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula, 
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. 

In [102]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        a = super(ClassNLLCriterion, self)
        super(ClassNLLCriterion, self).__init__()
        
    def updateOutput(self, input, target): 
        
        # Use this trick to avoid numerical errors
        eps = 1e-15 
        input_clamp = np.clip(input, eps, 1 - eps)
        
        # Your code goes here. ################################################
        self.output = -np.multiply(target, np.log(input_clamp)).sum() / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        
        # Use this trick to avoid numerical errors
        input_clamp = np.maximum(1e-15, np.minimum(input, 1 - 1e-15) )
                
        # Your code goes here. ################################################
        self.gradInput = -target / (input * input.shape[0])
        return self.gradInput
    
    def __repr__(self):
        return "ClassNLLCriterion"